### find a city

In [1]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

In [2]:
# Query to list all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

             name
0     goalscorers
1       shootouts
2       locations
3       countries
4        stations
5  noaa_countries
6         results
7         climate


### extract data for desired cities

In [4]:
pd.read_sql_query('''
SELECT count(distinct location_date_id) as count
FROM climate
''', conn)

,count
0,12539


In [55]:
pd.read_sql_query('''
SELECT location_date_id, count(*) as count
FROM climate
GROUP BY location_date_id
HAVING count > 1
order by count desc
limit 10
''', conn)

,location_date_id,count
0,CITY:JA000016_1958-05-28,1300
1,CITY:US360019_1949-06-19,1166
2,CITY:MX000020_1977-10-12,994
3,CITY:US080004_2019-06-19,896
4,CITY:AS000010_1973-03-16,756
5,CITY:AS000010_1973-03-13,756
6,CITY:AS000010_1973-03-11,752
7,CITY:AS000010_1973-03-24,748
8,CITY:MX000020_1968-03-10,705
9,CITY:MX000020_1977-10-23,656


In [5]:
pd.read_sql_query('''
SELECT count(distinct location_date_id) as count
FROM results
''', conn)

,count
0,28553


In [56]:
pd.read_sql_query('''
SELECT location_date_id, count(*) as count
FROM results
GROUP BY location_date_id
HAVING count > 1
order by count desc
limit 10
''', conn)

,location_date_id,count
0,None,16221
1,CITY:SG000001_1963-04-14,8
2,CITY:SG000001_1963-04-12,8
3,CITY:SG000001_1963-04-16,7
4,CITY:IV000002_1961-12-25,5
5,CITY:CF000001_1965-07-20,5
6,CITY:TS000012_1975-12-21,4
7,CITY:TH000005_1995-12-12,4
8,CITY:TH000005_1995-12-10,4
9,CITY:TH000005_1995-12-08,4


In [8]:
pd.read_sql_query('''
with part1 as(
select location_date_id,
max(case when datatype not in('err','err2') then 1 else 0 end) as valid,
max(case when datatype = 'err' then 1 else 0 end) as err_type1,
max(case when datatype = 'err2' then 1 else 0 end) as err_type2
from climate
group by 1
)
select sum(valid) as valid, sum(err_type1) as err_type1, sum(err_type2) as err_type2
from part1
''', conn)

,valid,err_type1,err_type2
0,20174,0,0


In [7]:
pd.read_sql_query('''
select count(distinct location_date_id)
FROM results
WHERE
location_id IS NOT NULL
and valid = 1
and location_date_id not in(select location_date_id from climate)
''', conn)

,count(distinct location_date_id)
0,0


In [6]:
# Create a DataFrame from the results table with locationid
test = pd.read_sql_query('''
SELECT distinct date_ts, location_id
FROM results
WHERE
location_id IS NOT NULL
and valid = 1
and location_date_id not in(select location_date_id from climate)
order by date_ts desc
limit 100
''', conn)
print(f"Number of rows in test: {len(test)}")

Number of rows in test: 0


In [23]:
import requests
import sqlite3
import json

# Define the endpoint
endpoint = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

# Set the headers with the API token
headers = {
    'token': api_token
}

# Define the parameters for the request
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily
    'locationid': '',
    'startdate': '',
    'enddate': '',
    'limit': 1000  # Number of results to return
}

In [ ]:
import time
import time
start_time = time.time()
iteration = 0
sucs = 0
fail = 0
# Iterate over the rows
for index, row in test.iterrows():
        # print('success: ', sucs, '/ fail: ', fail)
        iteration = iteration + 1
        # print(f"Iteration: {iteration}")
        if time.time() - start_time > 1800:  # 30 minutes
                print("Time limit exceeded. Stopping the process.")
                print('number of iterations: ', iteration)
                print('number of success: ', sucs)
                print('number of fail: ', fail)
                break
        params['locationid'] = row['location_id']
        params['startdate'] = row['date_ts']
        params['enddate'] = row['date_ts']
        # print(params)

        # Make the request
        try:
                response = requests.get(endpoint, headers=headers, params=params, timeout=30)
                try:
                        data = response.json()
                        # Check if 'results' key exists in the response data
                        if 'results' in data:
                                sucs = sucs + 1
                                # Insert data into the climate table
                                for record in data['results']:
                                        conn.execute('''
                                        INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                                        VALUES (?, ?, ?, ?, ?, ?)
                                        ''', (record['date'], row['location_id'], record['datatype'], record['station'], record['attributes'], record['value']))
                        else:
                                fail = fail + 1
                                # print(f"No results for location_id: {row['location_id']} on date: {row['date_ts']}")
                                continue
                                # conn.execute('''
                                # INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                                # VALUES (?, ?, ?, ?, ?, ?)
                                # ''', (row['date_ts'], row['location_id'], 'err', 'err', 'err', 'err'))
                        
                        conn.commit()

                except json.JSONDecodeError as e:
                        fail = fail + 1
                        # print(f"Error decoding JSON for location_id: {row['location_id']} on date: {row['date_ts']}")
                        continue
                #     raise e
                #     conn.execute('''
                #     INSERT OR REPLACE INTO climate (date, location_id, datatype, station, attributes, value)
                #     VALUES (?, ?, ?, ?, ?, ?)
                #     ''', (row['date_ts'], row['location_id'], 'err2', 'err2', 'err2', 'err2'))
                #     continue
        except requests.exceptions.Timeout:
                fail = fail + 1
                continue
        conn.commit()

conn.commit()

### create new column for location-date index

In [ ]:
print('number of iterations: ', iteration)
print('number of success: ', sucs)
print('number of fail: ', fail)

In [17]:
conn.execute('ALTER TABLE climate DROP COLUMN location_date_id')
conn.execute('ALTER TABLE climate ADD COLUMN location_date_id TEXT')
conn.execute('''
UPDATE climate
SET location_date_id = location_id || '_' || date(date)
''')
conn.commit()

### climate data coverage test

In [3]:
pd.read_sql_query('''
with part1 as(
select distinct date, location_id, location_date_id
from results
),
part2 as(
select distinct location_date_id, 1 as climate
from climate
),
part3 as(
select date(t1.date) as date, t1.location_id, t2.climate
from part1 t1
left join part2 t2
on t1.location_date_id = t2.location_date_id
)
select count(*), sum(climate) as climate_count
from part3
-- where climate is not null
limit 3
''', conn)

,count(*),climate_count
0,36555,20173


### climate data exploration

In [26]:
pd.read_sql_query(''' select * from climate limit 3''', conn)

,date,location_id,datatype,station,attributes,value,location_date_id
0,2024-01-01T00:00:00,CITY:JA000016,PRCP,GHCND:JA000047662,",,S,",0.0,CITY:JA000016_2024-01-01
1,2024-01-01T00:00:00,CITY:JA000016,TAVG,GHCND:JA000047662,"H,,S,",74.0,CITY:JA000016_2024-01-01
2,2024-01-01T00:00:00,CITY:JA000016,PRCP,GHCND:JA000047670,"B,,S,",0.0,CITY:JA000016_2024-01-01


In [30]:
pd.read_sql_query(''' select datatype, count(*) as freq from climate group by 1 order by 2 desc''', conn)

,datatype,freq
0,PRCP,158135
1,TMAX,74854
2,TMIN,74307
3,SNWD,39837
4,TAVG,26455
...,...,...
82,WT06,1
83,SX03,1
84,SN03,1
85,MDWM,1


In [44]:
pd.read_sql_query(''' select * from climate where location_id = 'CITY:CU000005'
order by date desc
limit 10 ''', conn)

,date,location_id,datatype,station,attributes,value,location_date_id
0,2016-10-07T00:00:00,CITY:CU000005,PRCP,GHCND:CUM00078325,",,S,",25.0,CITY:CU000005_2016-10-07
1,2016-10-07T00:00:00,CITY:CU000005,TAVG,GHCND:CUM00078325,"H,,S,",289.0,CITY:CU000005_2016-10-07
2,2016-10-07T00:00:00,CITY:CU000005,TMAX,GHCND:CUM00078325,",,S,",313.0,CITY:CU000005_2016-10-07
3,2016-10-07T00:00:00,CITY:CU000005,TMIN,GHCND:CUM00078325,",,S,",250.0,CITY:CU000005_2016-10-07
4,2015-06-14T00:00:00,CITY:CU000005,PRCP,GHCND:CUM00078325,",,S,",89.0,CITY:CU000005_2015-06-14
5,2015-06-14T00:00:00,CITY:CU000005,TAVG,GHCND:CUM00078325,"H,,S,",288.0,CITY:CU000005_2015-06-14
6,2015-06-14T00:00:00,CITY:CU000005,TMAX,GHCND:CUM00078325,",,S,",313.0,CITY:CU000005_2015-06-14
7,2015-06-14T00:00:00,CITY:CU000005,TMIN,GHCND:CUM00078325,",,S,",243.0,CITY:CU000005_2015-06-14
8,2012-10-16T00:00:00,CITY:CU000005,PRCP,GHCND:CUM00078325,",,S,",10.0,CITY:CU000005_2012-10-16
9,2012-10-16T00:00:00,CITY:CU000005,TAVG,GHCND:CUM00078325,"H,,S,",268.0,CITY:CU000005_2012-10-16


In [ ]:
pd.read_sql_query(''' select * from results where city like '%mosc%'
limit 10 ''', conn)

In [53]:
pd.read_sql_query('''
SELECT 
strftime('%m', date) as month,
avg(value) as avg_temp
FROM climate
WHERE
location_id = 'CITY:RS000038'
and datatype = 'PRCP'
GROUP BY 1
ORDER BY 1
''', conn)

,month,avg_temp
0,03,3.142857
1,04,0.000000
2,05,51.666667
3,06,40.421053
4,07,41.600000
5,08,36.833333
6,09,27.588235
7,10,17.125000
8,11,31.666667


In [ ]:
pd.read_sql_query('''
select location_id,
count(distinct station) as station_count,
count(distinct datatype) as type_count
from climate
group by 1
order by 2 desc
''', conn)

,location_id,station_count,type_count
0,CITY:US080004,562,31
1,CITY:US170006,352,55
2,CITY:US000001,321,50
3,CITY:US480057,309,39
4,CITY:AS000010,309,11
...,...,...,...
565,CITY:AG000006,1,4
566,CITY:AG000003,1,4
567,CITY:AG000002,1,4
568,CITY:AF000007,1,4


### merging climate data into results table

In [55]:
# Drop columns if they exist
conn.execute('ALTER TABLE results DROP COLUMN avg_tmax')
conn.execute('ALTER TABLE results DROP COLUMN avg_tmin')
conn.execute('ALTER TABLE results DROP COLUMN avg_tavg')
conn.execute('ALTER TABLE results DROP COLUMN avg_prcp')
conn.execute('ALTER TABLE results DROP COLUMN avg_snow')
conn.execute('ALTER TABLE results DROP COLUMN avg_snwd')
conn.execute('ALTER TABLE results DROP COLUMN avg_wt01')
conn.execute('ALTER TABLE results DROP COLUMN avg_wt02')

In [56]:

# Add new columns to the results table
conn.execute('ALTER TABLE results ADD COLUMN avg_tmax REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_tmin REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_tavg REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_prcp REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_snow REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_snwd REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_wt01 REAL')
conn.execute('ALTER TABLE results ADD COLUMN avg_wt02 REAL')
conn.commit()

In [57]:
# Execute the query and store the result in a DataFrame
climate_averages = pd.read_sql_query('''
select location_date_id,
avg(case when datatype = 'TMAX' then value / 10 else null end) as avg_tmax,
avg(case when datatype = 'TMIN' then value / 10 else null end) as avg_tmin,
avg(case when datatype = 'TAVG' then value / 10 else null end) as avg_tavg,
avg(case when datatype = 'PRCP' then value else null end) as avg_prcp,
avg(case when datatype = 'SNOW' then value else null end) as avg_snow,
avg(case when datatype = 'SNWD' then value else null end) as avg_snwd,
avg(case when datatype = 'WT01' then value else null end) as avg_wt01,
avg(case when datatype = 'WT02' then value else null end) as avg_wt02
from climate
group by 1
''', conn)

# Update the results table with the calculated averages
for index, row in climate_averages.iterrows():
    conn.execute('''
    UPDATE results
    SET avg_tmax = ?, avg_tmin = ?, avg_tavg = ?, avg_prcp = ?, avg_snow = ?, avg_snwd = ?, avg_wt01 = ?, avg_wt02 = ?
    WHERE location_date_id = ?
    ''', (row['avg_tmax'], row['avg_tmin'], row['avg_tavg'], row['avg_prcp'], row['avg_snow'], row['avg_snwd'], row['avg_wt01'], row['avg_wt02'], row['location_date_id']))

conn.commit()

### test the update

In [39]:
pd.read_sql_query('''
select * from results
where valid = 1
limit 3
''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,...,valid,location_date_id,avg_tmax,avg_tmin,avg_tavg,avg_prcp,avg_snow,avg_snwd,avg_wt01,avg_wt02
0,26,1883-03-10 00:00:00,England,Scotland,2,3,Friendly,Sheffield,United Kingdom,0,...,1,CITY:UK000011_1883-03-10,22.0,-84.0,None,0.0,None,None,None,None
1,49,1887-02-05 00:00:00,England,Northern Ireland,7,0,British Home Championship,Sheffield,United Kingdom,0,...,1,CITY:UK000011_1887-02-05,112.0,51.0,None,0.0,None,None,None,None
2,86,1893-02-25 00:00:00,England,Northern Ireland,6,1,British Home Championship,Birmingham,United Kingdom,0,...,1,CITY:UK000002_1893-02-25,55.0,-15.0,None,NaN,None,None,None,None


In [42]:
pd.read_sql_query('''
select
count(*) as count_all,
sum(case when valid = 1 then 1 else 0 end) as count_valid,
sum(case when valid = 1 and (avg_tmax is not null or avg_tmin is not null) then 1 else 0 end) as count_valid_and_temp,
sum(case when valid = 1 and (
avg_tmax is not null or
avg_tmin is not null or
avg_tavg is not null or
avg_prcp is not null or
avg_snow is not null or
avg_snwd is not null or
avg_wt01 is not null or
avg_wt02 is not null
) then 1 else 0 end) as count_valid_and_clim
from results
''', conn)

,count_all,count_valid,count_valid_and_temp,count_valid_and_clim
0,47598,28471,18906,22023


In [ ]:
pd.read_sql_query('''